In [ ]:
using CSV
using DataFrames
using Serialization
using HDF5
using ADCME
using ADTomo
using PyCall
using Dates
using PyPlot
using Random
using Base
using LinearAlgebra
Random.seed!(233)
reset_default_graph()

In [ ]:
m = 40           #width
n = 30           #length
h = 1.0          #resolution

In [ ]:
f = ones(n,m)
f[16:20,20:24] .= 2                #design velocity model

In [ ]:
allsrc = DataFrame(x = [], y = [])
allrcv = DataFrame(x = [], y = [])

for rcv_x = 5:8:m
    for rcv_y = 5:8:n
        push!(allrcv.x,rcv_x)
        push!(allrcv.y,rcv_y)
    end
end                                 #design the locations of stations

for i = 1:40                        #the number of events
    push!(allsrc.x,rand(1:m))
    push!(allsrc.y,rand(1:n))
end                                 #design the locations of events

numrcv = size(allrcv,1)
numsrc = size(allsrc,1)

In [ ]:
u = PyObject[]
for i=1:numsrc
    push!(u,eikonal(f,allsrc.x[i],allsrc.y[i],h))
end
sess = Session()
init(sess)
uobs = run(sess, u)                                      #uobs is a list of [numsrc * (m,n)] representing travel time

In [ ]:
obs_time = Array{Float64}(undef,numsrc,numrcv)
for i = 1:numsrc
    for j = 1:numrcv
        obs_time[i,j] = uobs[i][allrcv.y[j],allrcv.x[j]]
    end
end

In [ ]:
i = 5                                      #choose a source to plot a traveltime image
figure()
pcolormesh(uobs[i], cmap = "Purples")
colorbar()
scatter(allsrc.x[i],allsrc.y[i])
title("traveltime_image_of_source_$i")
savefig("traveltime_$i.png")

In [ ]:
fvar = Variable(ones(n, m))                          #design an original velocity model for inversion
u = PyObject[]
for i=1:numsrc
    push!(u,eikonal(fvar,allsrc.x[i],allsrc.y[i],h))
end

In [49]:
loss = sum([sum((obs_time[i,j] - u[i][allrcv.y[j],allrcv.x[j]])^2) for i = 1:numsrc for j=1:numrcv])
init(sess)
@show run(sess, loss)
BFGS!(sess, loss, 200, var_to_bounds=Dict(fvar=>(0.5,3.0)))                                          #200 means max iteration steps and (0.5,100.0) means velocity change range

In [ ]:
figure(figsize=(10, 4))
subplot(121)
pcolormesh(f)
colorbar()
title("True")
scatter(allsrc.x,allsrc.y,label="event")
scatter(allrcv.x,allrcv.y,label="station")
legend()
subplot(122)
pcolormesh(run(sess,fvar),vmin=0.8,vmax=2.2)                #vmin & vmax
colorbar()
title("Inverted")
savefig("inversion_result.png")